__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [18]:
#
## Your code for initializing the transformer model(s)
#
# Note that transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import *
import transformers

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

loading file vocab.json from cache at C:\Users\Cyproxius/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\vocab.json
loading file merges.txt from cache at C:\Users\Cyproxius/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\Cyproxius/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {


In [19]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/file/d/19Lp3AM4NEPycp_IBgoHfLc_V456pmUom/view?usp=sharing
# You can read more about this model in the original paper here: https://arxiv.org/pdf/1803.11138.pdf
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel
import torch


model_location = 'state_dict.pt'  # <- point this to the location of the Gulordava .pt file
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt', encoding="utf8") as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)

It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [20]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses


ud_parses = parse_corpus('data/sample/en_ewt-ud-train.conllu')

In [21]:
# Single sentence
ud_parses[0]

TokenList<Al, -, Zaman, :, American, forces, killed, Shaikh, Abdullah, al, -, Ani, ,, the, preacher, at, the, mosque, in, the, town, of, Qaim, ,, near, the, Syrian, border, ., metadata={newdoc id: "weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000", sent_id: "weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000-0001", text: "Al-Zaman : American forces killed Shaikh Abdullah al-Ani, the preacher at the mosque in the town of Qaim, near the Syrian border."}>

In [22]:
for token in ud_parses[0]:
    print(token['misc'])

{'SpaceAfter': 'No'}
{'SpaceAfter': 'No'}
None
None
None
None
None
None
None
{'SpaceAfter': 'No'}
{'SpaceAfter': 'No'}
{'SpaceAfter': 'No'}
None
None
None
None
None
None
None
None
None
None
{'SpaceAfter': 'No'}
None
None
None
None
{'SpaceAfter': 'No'}
None


In [23]:
# Single token
ud_parses[0][0]

{'id': 1,
 'form': 'Al',
 'lemma': 'Al',
 'upos': 'PROPN',
 'xpos': 'NNP',
 'feats': {'Number': 'Sing'},
 'head': 0,
 'deprel': 'root',
 'deps': [('root', 0)],
 'misc': {'SpaceAfter': 'No'}}

In [24]:
# Obtain pos tag of token
ud_parses[0][0]['xpos']

'NNP'

In [25]:
# Tree of sentence
ud_parses[0].to_tree()

TokenTree<token={id=1, form=Al}, children=[...]>

In [26]:
# Print tree of sentence
ud_parses[0].to_tree().print_tree()

(deprel:root) form:Al lemma:Al upos:PROPN [1]
    (deprel:punct) form:- lemma:- upos:PUNCT [2]
    (deprel:flat) form:Zaman lemma:Zaman upos:PROPN [3]
    (deprel:punct) form:: lemma:: upos:PUNCT [4]
    (deprel:parataxis) form:killed lemma:kill upos:VERB [7]
        (deprel:nsubj) form:forces lemma:force upos:NOUN [6]
            (deprel:amod) form:American lemma:american upos:ADJ [5]
        (deprel:obj) form:Shaikh lemma:Shaikh upos:PROPN [8]
            (deprel:flat) form:Abdullah lemma:Abdullah upos:PROPN [9]
            (deprel:flat) form:al lemma:al upos:PROPN [10]
            (deprel:punct) form:- lemma:- upos:PUNCT [11]
            (deprel:flat) form:Ani lemma:Ani upos:PROPN [12]
            (deprel:punct) form:, lemma:, upos:PUNCT [13]
            (deprel:appos) form:preacher lemma:preacher upos:NOUN [15]
                (deprel:det) form:the lemma:the upos:DET [14]
        (deprel:obl) form:mosque lemma:mosque upos:NOUN [18]
            (deprel:case) form:at lemma:at upos:AD

# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
1. Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".


2. Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.


3. **Important**: If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. <br /><br />
Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. That is, the tokenizer uses a different token id for `"man"`, than it does for `" man"`: the former could be part of `" woman"`=`" wo`"+`"man"`, whereas the latter would be the used in case *man* occurs at the start of a word. The tokenizer for GPT-2 adds spaces at the start of a token (represented as a `Ġ` symbol). This means that you should keep track whether the previous token had the `SpaceAfter` attribute set to `'No'`: in case it did not, you should manually prepend a `" "` ahead of the token.


4. The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.


5. The huggingface transformer models don't return the hidden state by default. To achieve this you can pass `output_hidden_states=True` to a model forward pass. The hidden states are then returned for all intermediate layers as well, the latest entry in this list corresponds to the top layer.


6. **N.B.**: Make sure that when you run a sentence through your model, you do so within a `with torch.no_grad():` block, and that you have run `model.eval()` beforehand as well (to disable dropout).


7. **N.B.**: Make sure to use a token's ``["form"]`` attribute, and not the ``["lemma"]``, as the latter will stem any relevant morphological information from the token. We don't want this, because we want to feed well-formed, grammatical sentences to our model.


I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [27]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle


# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!

def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer, concat=True, layer=-1) -> Tensor:
    
    model.eval()
    # Initialize an empty list to store sentence representations
    corpus_rep = []
    
    # Transformer model
    if isinstance(tokenizer, transformers.PreTrainedTokenizer):
        # Loop through the parses
        for parse in ud_parses:
            prev_token = None
            encoded_full = []
            encoded = []
            tokens = []
            # Loop through the tokens in the parse
            for token in parse:
                # Check if the previous token has a misc attribute and SpaceAfter key
                if prev_token is None or ('misc' in prev_token and prev_token['misc'] is not None and 'SpaceAfter' in prev_token['misc'] and prev_token['misc']['SpaceAfter'] == 'No'):
                    tokens.append(token['form'])
                else:
                    tokens.append(' ' + token['form'])
                prev_token = token

            # Encode each word using the tokenizer
            for token in tokens:
                with torch.no_grad():
                    encoded_token = tokenizer.encode(token, add_special_tokens=False)
                    encoded.extend(encoded_token)
                    encoded_full.append(encoded_token)

            # Convert the encoded subword to a tensor        
            input_ids = torch.tensor(encoded).unsqueeze(0)
            # Get the sentence representation from the model's output
            with torch.no_grad():
                outputs = model(input_ids, output_hidden_states=True)

            last_layer_hidden_state = outputs.hidden_states[layer]
            sen_rep = last_layer_hidden_state.squeeze()
            # Fix for one token sentences
            if len(parse) <= 1:
                sen_rep = sen_rep.unsqueeze(0)
            # Average the representations of subwords
            sen_rep_avg = []
            i = 0
            for word in encoded_full:
                j = len(word)
                sen_rep_avg.append(torch.mean(sen_rep[i:i+j], dim=0, keepdim=True))
                i += j
            tensor = torch.stack(sen_rep_avg, dim=0).squeeze(1)
            if tensor.shape[0] == 1:
                tensor = tensor.view(1, -1, tensor.shape[-1])[:, 0, :]      
            corpus_rep.append(tensor)
        
    # LSTM model
    else:
        for parse in ud_parses:
            encoded = [tokenizer.get(token['form'].replace(' ', ''), tokenizer['<unk>']) for token in parse]
            input_ids = torch.tensor(encoded).unsqueeze(0)
            hidden = model.init_hidden(1)
            with torch.no_grad():
                tensor = lstm(input_ids, hidden).squeeze()
            if len(tensor.shape) == 1:
                tensor = tensor.unsqueeze(0)
            corpus_rep.append(tensor)
    if concat:
        corpus_rep = torch.cat(corpus_rep, dim=0)
    
    return corpus_rep


To validate your activation extraction procedure I have set up the following assertion function as a sanity check. It compares your representation of the first sentence in the corpus against a pickled version of mine. 

For this I used `distilgpt2`.

In [28]:
def error_msg(model_name, gold_embs, embs, i2w):
    with open(f'{model_name}_tokens1.pickle', 'rb') as f:
        sen_tokens = pickle.load(f)
        
    diff = torch.abs(embs - gold_embs)
    max_diff = torch.max(diff)
    avg_diff = torch.mean(diff)
    
    print(f"{model_name} embeddings don't match!")
    print(f"Max diff.: {max_diff:.4f}\nMean diff. {avg_diff:.4f}")

    print("\nCheck if your tokenization matches with the original tokenization:")
    for idx in sen_tokens.squeeze():
        if isinstance(i2w, list):
            token = i2w[idx]
        else:
            token = i2w.convert_ids_to_tokens(idx.item())
        print(f"{idx:<6} {token}")


def assert_sen_reps(model, tokenizer, lstm, vocab):
    with open('distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)
        
    with open('lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
    
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, model, tokenizer)
    own_lstm_emb1 = fetch_sen_reps(corpus, lstm, vocab)
    
    assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape, \
        f"Distilgpt2 shape mismatch: {distilgpt2_emb1.shape} (gold) vs. {own_distilgpt2_emb1.shape} (yours)"
    assert lstm_emb1.shape == own_lstm_emb1.shape, \
        f"LSTM shape mismatch: {lstm_emb1.shape} (gold) vs. {own_lstm_emb1.shape} (yours)"

    if not torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1, rtol=1e-3, atol=1e-3):
        error_msg("distilgpt2", distilgpt2_emb1, own_distilgpt2_emb1, tokenizer)
    if not torch.allclose(lstm_emb1, own_lstm_emb1, rtol=1e-3, atol=1e-3):
        error_msg("lstm", lstm_emb1, own_lstm_emb1, list(vocab.keys()))


assert_sen_reps(model, tokenizer, lstm, vocab)

Next, we should define a function that extracts the corresponding POS labels for each activation, which we do based on the **``"upostag"``** attribute of a token (so not the ``xpostag`` attribute). These labels will be transformed to a tensor containing the label index for each item.

In [29]:
# FETCH POS LABELS
from typing import Optional, Dict, Tuple

# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], pos_vocab: Optional[Dict[str, int]] = None) -> Tuple[Tensor, Dict[str, int]]:
    
    # Create label dictionary if not passed to function
    if pos_vocab is None:
        pos_vocab = {}
        pos_counter = 0
    else:
        pos_counter = len(pos_vocab.values())

    # Keep a list of all POS tag labels
    all_tags = []

    for parse in ud_parses:
        for token in parse:
            # Extract the POS tag for the given token
            pos_tag = token['upostag']
            
            # If the token is not yet in the vocabulary, add it
            if pos_tag not in pos_vocab.keys():
                pos_vocab[pos_tag] = pos_counter
                pos_counter += 1
            
            # Add the label encoding of the POS tag
            all_tags.append(pos_vocab[pos_tag])

    return torch.tensor(all_tags), pos_vocab


fetch_pos_tags(ud_parses)

(tensor([0, 1, 0,  ..., 3, 0, 4]),
 {'PROPN': 0,
  'PUNCT': 1,
  'ADJ': 2,
  'NOUN': 3,
  'VERB': 4,
  'DET': 5,
  'ADP': 6,
  'AUX': 7,
  'PRON': 8,
  'PART': 9,
  'SCONJ': 10,
  'NUM': 11,
  'ADV': 12,
  'CCONJ': 13})

Finally, we can combine all these methods to retrieve the representations (`fetch_sen_reps`) and the corresponding labels (`fetch_pos_tags`). If you are still debugging and testing your setup you can set the `use_sample` variable to `True`, and once everything works you can extract the full corpus by setting it to `False`.

The reason we pass the `train_vocab` to the data creation of the `dev` and `test` data is that we want to use the same label vocabulary across the different train/dev/test splits.

In [30]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(ud_parses, filename: str, lm, w2i, pos_vocab=None, layer=-1):
    ud_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(ud_parses, lm, w2i, layer=layer)
    pos_tags, pos_vocab = fetch_pos_tags(ud_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab

## Get data: Sample data, useful for testing

**This cell is used to generate the representations for all distilgpt2 layers. This needs to be run only once!**

In [31]:
# import time

# lm = model  # or `lstm`
# w2i = tokenizer  # or `vocab`
# use_sample = True

# # distilgpt2 has 7 hidden layers
# num_layers = 7

# # Generate data for every hidden layer
# for layer in range(num_layers):

#     print(f"Generating representations for layer {layer}")

#     t = time.time()
#     print(f"Generating training data...")
#     train_x, train_y, train_vocab = create_data(
#         ud_parses,
#         os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
#         lm, 
#         w2i,
#         layer=layer
#     )
#     print(f"Done generating training data. This took {time.time() - t}s")

#     t = time.time()
#     print(f"Generating validation data...")
#     dev_x, dev_y, _ = create_data(
#         ud_parses,
#         os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
#         lm, 
#         w2i,
#         pos_vocab=train_vocab,
#         layer=layer
#     )
#     print(f"Done generating validation data. This took {time.time() - t}s")

#     t = time.time()
#     print(f"Generating test data...")
#     test_x, test_y, _ = create_data(
#         ud_parses,
#         os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
#         lm,
#         w2i,
#         pos_vocab=train_vocab,
#         layer=layer
#     )
#     print(f"Done generating test data. This took {time.time() - t}s")

#     print(f"Now writing all data to pickle")
#     with open(f'pickles/GPT/train_x_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(train_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(f'pickles/GPT/train_y_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(train_y, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(f'pickles/GPT/train_vocab_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(train_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#     with open(f'pickles/GPT/dev_x_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(dev_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(f'pickles/GPT/dev_y_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(dev_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     with open(f'pickles/GPT/test_x_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(test_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(f'pickles/GPT/test_y_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(test_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     train_x_big = torch.cat((train_x, dev_x), dim=0)
#     train_y_big = torch.cat((train_y, dev_y), dim=0)

#     with open(f'pickles/GPT/train_x_big_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(train_x_big, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(f'pickles/GPT/train_y_big_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(train_y_big, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     print(f"Done writing data to pickle")
#     # Delete all lists so memory doesn't explode
#     del train_x
#     del train_y
#     del train_vocab
#     del dev_x
#     del dev_y
#     del test_x
#     del test_y
#     del train_x_big
#     del train_y_big

In [32]:
# lm = model  # or `lstm`
# w2i = tokenizer  # or `vocab`
# use_sample = True

# train_x, train_y, train_vocab = create_data(
#     ud_parses,
#     os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
#     lm, 
#     w2i
# )

# dev_x, dev_y, _ = create_data(
#     ud_parses,
#     os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
#     lm, 
#     w2i,
#     pos_vocab=train_vocab
# )

# test_x, test_y, _ = create_data(
#     ud_parses,
#     os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
#     lm,
#     w2i,
#     pos_vocab=train_vocab
# )

## Get data: Whole dataset

### Generate

In [33]:
# lm = lstm  # or `lstm`
# w2i = vocab  # or `vocab`
# use_sample = False

# train_x, train_y, train_vocab = create_data(
#     ud_parses,
#     os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
#     lm, 
#     w2i
# )

# dev_x, dev_y, _ = create_data(
#     ud_parses,
#     os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
#     lm, 
#     w2i,
#     pos_vocab=train_vocab
# )

# test_x, test_y, _ = create_data(
#     ud_parses,
#     os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
#     lm,
#     w2i,
#     pos_vocab=train_vocab
# )

### Pickle dump

In [34]:
# with open('train_x.pickle', 'wb') as handle:
#     pickle.dump(train_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('train_y.pickle', 'wb') as handle:
#     pickle.dump(train_y, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('train_vocab.pickle', 'wb') as handle:
#     pickle.dump(train_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('dev_x.pickle', 'wb') as handle:
#     pickle.dump(dev_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('dev_y.pickle', 'wb') as handle:
#     pickle.dump(dev_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_x.pickle', 'wb') as handle:
#     pickle.dump(test_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('test_y.pickle', 'wb') as handle:
#     pickle.dump(test_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
# train_x_big = torch.cat((train_x, dev_x), dim=0)
# train_y_big = torch.cat((train_y, dev_y), dim=0)

In [36]:
# with open('train_x_big.pickle', 'wb') as handle:
#     pickle.dump(train_x_big, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('train_y_big.pickle', 'wb') as handle:
#     pickle.dump(train_y_big, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Pickle load

In [37]:
# GPT or lstm
model_name = "GPT"

# Load data from pickle
with open('pickles/' + model_name + '/train_x.pickle', 'rb') as handle:
    GPT_train_x = torch.load(handle)
with open('pickles/' + model_name + '/train_y.pickle', 'rb') as handle:
    GPT_train_y = torch.load(handle)
with open('pickles/' + model_name + '/train_vocab.pickle', 'rb') as handle:
    GPT_train_vocab = torch.load(handle)

with open('pickles/' + model_name + '/dev_x.pickle', 'rb') as handle:
    GPT_dev_x = torch.load(handle)
with open('pickles/' + model_name + '/dev_y.pickle', 'rb') as handle:
    GPT_dev_y = torch.load(handle)

with open('pickles/' + model_name + '/test_x.pickle', 'rb') as handle:
    GPT_test_x = torch.load(handle)
with open('pickles/' + model_name + '/test_y.pickle', 'rb') as handle:
    GPT_test_y = torch.load(handle)

# Big train set that is dev set and train set combined, as skorch does train/dev split itself
with open('pickles/' + model_name + '/train_x_big.pickle', 'rb') as handle:
    GPT_train_x_big = torch.load(handle)
with open('pickles/' + model_name + '/train_y_big.pickle', 'rb') as handle:
    GPT_train_y_big = torch.load(handle)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# GPT or lstm
model_name = "lstm"

# Load data from pickle
with open('pickles/' + model_name + '/train_x.pickle', 'rb') as handle:
    lstm_train_x = pickle.load(handle)
with open('pickles/' + model_name + '/train_y.pickle', 'rb') as handle:
    lstm_train_y = pickle.load(handle)
with open('pickles/' + model_name + '/train_vocab.pickle', 'rb') as handle:
    lstm_train_vocab = pickle.load(handle)

with open('pickles/' + model_name + '/dev_x.pickle', 'rb') as handle:
    lstm_dev_x = pickle.load(handle)
with open('pickles/' + model_name + '/dev_y.pickle', 'rb') as handle:
    lstm_dev_y = pickle.load(handle)

with open('pickles/' + model_name + '/test_x.pickle', 'rb') as handle:
    lstm_test_x = pickle.load(handle)
with open('pickles/' + model_name + '/test_y.pickle', 'rb') as handle:
    lstm_test_y = pickle.load(handle)

# Big train set that is dev set and train set combined, as skorch does train/dev split itself
with open('pickles/' + model_name + '/train_x_big.pickle', 'rb') as handle:
    lstm_train_x_big = pickle.load(handle)
with open('pickles/' + model_name + '/train_y_big.pickle', 'rb') as handle:
    lstm_train_y_big = pickle.load(handle)

# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

### Simple classifier using sklearn
> If you set train_x etc. to the sample data you can test this out yourself. I merge train_x and dev_x because sklearn makes a train/dev split itself using k-fold.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the Logistic Regression model
model = LogisticRegression()

In [ ]:
# GPT
# Training phase
model.fit(GPT_train_x, GPT_train_y)

# Predict using the trained model
GPT_train_predictions = model.predict(GPT_train_x)
GPT_dev_predictions = model.predict(GPT_dev_x)
GPT_test_predictions = model.predict(GPT_test_x)

# Calculate accuracy scores
GPT_train_accuracy = accuracy_score(GPT_train_y, GPT_train_predictions)
GPT_dev_accuracy = accuracy_score(GPT_dev_y, GPT_dev_predictions)
GPT_test_accuracy = accuracy_score(GPT_test_y, GPT_test_predictions)

print("Training accuracy:", GPT_train_accuracy)
print("Dev accuracy:", GPT_dev_accuracy)
print("Test accuracy:", GPT_test_accuracy)

In [ ]:
# LSTM
# Training phase
model.fit(lstm_train_x, lstm_train_y)

# Predict using the trained model
lstm_train_predictions = model.predict(lstm_train_x)
lstm_dev_predictions = model.predict(lstm_dev_x)
lstm_test_predictions = model.predict(lstm_test_x)

# Calculate accuracy scores
lstm_train_accuracy = accuracy_score(lstm_train_y, lstm_train_predictions)
lstm_dev_accuracy = accuracy_score(lstm_dev_y, lstm_dev_predictions)
lstm_test_accuracy = accuracy_score(lstm_test_y, lstm_test_predictions)

print("Training accuracy:", lstm_train_accuracy)
print("Dev accuracy:", lstm_dev_accuracy)
print("Test accuracy:", lstm_test_accuracy)

# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [ ]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [ ]:
# !pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

In [ ]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)

As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

In [ ]:
def create_gold_distances(corpus):
    all_distances = []

    for item in corpus:
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)

        sen_len = len(ete_tree.search_nodes())
        distances = torch.zeros((sen_len, sen_len))

        # Compute all the distances
        for i in range(sen_len):
            for j in range(sen_len):
                if i == j:
                    distances[i,j] = 0.0
                    distances[j,i] = 0.0
                else:
                    distances[i,j] = ete_tree.get_distance(str(i+1), str(j+1))
                    distances[j,i] = ete_tree.get_distance(str(i+1), str(j+1))

        all_distances.append(distances)

    return all_distances

The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

In [ ]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    
    return mst

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [ ]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]
print(gold_distance, '\n')

mst = create_mst(gold_distance)
print(mst)

In [ ]:
print(gold_distance, '\n')

Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [ ]:
def edges(mst):
    # Your code for retrieving the edges from the MST matrix
    edges = set()
    for i, node1 in enumerate(mst):
        for j, node2 in enumerate(node1):
            if node2 == 1:
                edges.add((i,j))
                edges.add((j,i))

    return edges

def calc_uuas(pred_distances, gold_distances):
    uuas = None
    # Find the indices of non-padded rows and columns
    non_padded_rows = torch.any(gold_distances != -1, dim=1)
    non_padded_cols = torch.any(gold_distances != -1, dim=0)

    # Slice both tensors using the non-padded indices
    pred_distances = pred_distances[non_padded_rows][:, non_padded_cols]
    gold_distances = gold_distances[non_padded_rows][:, non_padded_cols]
    
    pred_mst = create_mst(pred_distances)
    gold_mst = create_mst(gold_distances)
    pred_edges = edges(pred_mst)
    gold_edges = edges(gold_mst)
    correct = pred_edges.intersection(gold_edges)
    if len(gold_edges) == 0:
        uuas = 1
    else:
        uuas = len(correct) / len(gold_edges)
    return uuas


# Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

In [ ]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        self.proj_norms = nn.Parameter(data=torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        nn.init.uniform_(self.proj_norms, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        transformed_norms = torch.matmul(batch, self.proj_norms)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)
        
        squared_norms = transformed_norms.pow(2).sum(-1)

        return squared_distances, squared_norms

    
    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [ ]:
from torch import optim
import math

'''
Similar to the `create_data` method of the previous notebook, I recommend you to use a method 
that initialises all the data of a corpus. Note that for your embeddings you can use the 
`fetch_sen_reps` method again. However, for the POS probe you concatenated all these representations into 
1 big tensor of shape (num_tokens_in_corpus, model_dim). 

The StructuralProbe expects its input to contain all the representations of 1 sentence, so I recommend you
to update your `fetch_sen_reps` method in a way that it is easy to retrieve all the representations that 
correspond to a single sentence.
''' 

def init_corpus(path, concat=False, cutoff=None, layer=-1):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.
    """
    corpus = parse_corpus(path)[:cutoff]
    embs = fetch_sen_reps(corpus, lstm, vocab, concat=concat, layer=layer)
#     embs = fetch_sen_reps(corpus, model, tokenizer, concat=concat, layer=layer)    
    gold_distances = create_gold_distances(corpus)
    
    return gold_distances, embs


# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, _data):
    loss_function =  L1DistanceLoss()
    probe.eval()
    
    with torch.no_grad():
        distances = _data[0]
        embs = _data[1]
        
        # Make big tensor of shape (batch_size, max_seq_len, representation_dim)
        # from the representations in the batch using zero-padding
        max_seq_len = max([tensor.shape[0] for tensor in embs])
        batch_size = len(embs)
        representation_dim = embs[0].shape[1]
        embs_tensor = torch.zeros(batch_size, max_seq_len, representation_dim)
        for i, tensor in enumerate(embs):
            if tensor.shape[0] == 1:
                tensor = tensor.view(1, -1, representation_dim)[:, 0, :]
            seq_len = tensor.shape[0]
            embs_tensor[i, :seq_len] = tensor

        # Predict distances and norms using probe
        predicted_distances, predicted_norms = probe(embs_tensor)

        # Make big tensor of distances from batch padded with -1
        # resulting in true_distances of shape (batch_size, max_seq_len, max_seq_len)
        max_seq_len = max([tensor.shape[0] for tensor in distances])
        batch_size = len(distances)
        sentence_lengths = torch.zeros(batch_size, dtype=torch.long)
        true_distances = torch.full((batch_size, max_seq_len, max_seq_len), -1)
        for i, tensor in enumerate(distances):
            seq_len = tensor.shape[0]
            true_distances[i, :seq_len, :seq_len] = tensor
            sentence_lengths[i] = seq_len

        # Calculate loss and uass score
        loss, total_sents = loss_function(predicted_distances, true_distances, sentence_lengths)
        uuas_score = [calc_uuas(pred, true) for pred, true in zip(predicted_distances, true_distances)]
    
    return loss.item(), uuas_score


# Feel free to alter the signature of this method.
def train(_train_data, _dev_data, _test_data, epochs=1000):
    emb_dim = _train_data[1][0].shape[-1]
    rank = 64
    lr = 10e-4
    batch_size = 24
    dev_losses = []
    dev_uuases = []

    probe = StructuralProbe(emb_dim, rank)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    for epoch in range(epochs):
        print("Epoch:", epoch)
        for i in range(0, len(_train_data[1]), batch_size):
            optimizer.zero_grad()

            # YOUR CODE FOR DOING A PROBE FORWARD PASS
            batch_embs = _train_data[1][i:i+batch_size]
            batch_distances = _train_data[0][i:i+batch_size]
            
            
            # Make big tensor of shape (batch_size, max_seq_len, representation_dim)
            # from the representations in the batch using zero-padding
            max_seq_len = max([tensor.shape[0] for tensor in batch_embs])
            embs_tensor = torch.zeros(len(batch_embs), max_seq_len, emb_dim)
            for j, tensor in enumerate(batch_embs):
                if tensor.shape[0] == 1:
                    tensor = tensor.view(1, -1, emb_dim)[:, 0, :]
                seq_len = tensor.shape[0]
                embs_tensor[j, :seq_len] = tensor
    
            # Predict distances and using probe
            predicted_distances, predicted_norms = probe(embs_tensor)
            
            # Make big tensor of shape (batch_size, max_seq_len, max_seq_len) from distances
            # in the batch using padding with -1
            max_seq_len = max([tensor.shape[0] for tensor in batch_distances])
            sentence_lengths = torch.zeros(len(batch_embs), dtype=torch.long)
            true_distances = torch.full((len(batch_embs), max_seq_len, max_seq_len), -1)
            for j, tensor in enumerate(batch_distances):
                seq_len = tensor.shape[0]
                true_distances[j, :seq_len, :seq_len] = tensor
                sentence_lengths[j] = seq_len
            
            # Calculate loss
            batch_loss, total_sents = loss_function(predicted_distances, true_distances, sentence_lengths)

            batch_loss.backward()
            optimizer.step()

        dev_loss, dev_uuas = evaluate_probe(probe, _dev_data)
        dev_losses.append(dev_loss)
        dev_uuases.append(sum(dev_uuas)/len(dev_uuas))
        

        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe, _test_data)
    return dev_losses, dev_uuases, test_loss, sum(test_uuas)/len(test_uuas)

In [ ]:
train_data[1][0].shape

# Get data

### Sample data, useful for debugging

In [ ]:
# train_sample = init_corpus('data/sample/en_ewt-ud-train.conllu', concat=False, cutoff=None)
# dev_sample = init_corpus('data/sample/en_ewt-ud-dev.conllu', concat=False, cutoff=None)
# test_sample = init_corpus('data/sample/en_ewt-ud-test.conllu', concat=False, cutoff=None)

### Load full dataset for first, takes some time so I put it in a pickle

In [ ]:
# # Initialize the corpus
# train_data = init_corpus('data/en_ewt-ud-train.conllu', concat=False, cutoff=None)
# dev_data = init_corpus('data/en_ewt-ud-dev.conllu', concat=False, cutoff=None)
# test_data = init_corpus('data/en_ewt-ud-test.conllu', concat=False, cutoff=None)

### Dump data in pickle

In [ ]:
# with open('struct_train.pickle', 'wb') as handle:
#     pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('struct_dev.pickle', 'wb') as handle:
#     pickle.dump(dev_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('struct_test.pickle', 'wb') as handle:
#     pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Load data from pickle

In [ ]:
model_name = "GPT"

with open('pickles/' + model_name + '/struct_train.pickle', 'rb') as handle:
    GPT_train_data = pickle.load(handle)
with open('pickles/' + model_name + '/struct_dev.pickle', 'rb') as handle:
    GPT_dev_data = pickle.load(handle)
with open('pickles/' + model_name + '/struct_test.pickle', 'rb') as handle:
    GPT_test_data = pickle.load(handle)

In [ ]:
model_name = "lstm"

with open('pickles/' + model_name + '/struct_train.pickle', 'rb') as handle:
    lstm_train_data = pickle.load(handle)
with open('pickles/' + model_name + '/struct_dev.pickle', 'rb') as handle:
    lstm_dev_data = pickle.load(handle)
with open('pickles/' + model_name + '/struct_test.pickle', 'rb') as handle:
    lstm_test_data = pickle.load(handle)

In [ ]:
print(len(train_data[0]))
print(len(dev_data[0]))
print(len(train_data[0]))

**Another cell to create data for all hidden layers of the distilgpt2 model**

In [ ]:
# import time

# lm = model  # or `lstm`
# w2i = tokenizer  # or `vocab`
# use_sample = True

# # distilgpt2 has 7 hidden layers
# num_layers = 7

# # Generate data for every hidden layer
# for layer in range(num_layers):

#     print(f"Generating representations for layer {layer}")

#     t = time.time()
#     print(f"Generating training data...")
#     train_data = init_corpus('data/en_ewt-ud-train.conllu', concat=False, cutoff=None, layer=layer)
#     print(f"Done generating training data. This took {time.time() - t}s")

#     t = time.time()
#     print(f"Generating validation data...")
#     dev_data = init_corpus('data/en_ewt-ud-dev.conllu', concat=False, cutoff=None, layer=layer)
#     print(f"Done generating validation data. This took {time.time() - t}s")

#     t = time.time()
#     print(f"Generating test data...")
#     test_data = init_corpus('data/en_ewt-ud-test.conllu', concat=False, cutoff=None, layer=layer)
#     print(f"Done generating test data. This took {time.time() - t}s")

#     print(f"Now writing all data to pickle")

#     with open(f'pickles/GPT/struct_train_layer_{layer}.pickle', 'wb') as handle:
#         pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     with open(f'pickles/GPT/struct_dev.pickle_layer_{layer}', 'wb') as handle:
#         pickle.dump(dev_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     with open(f'pickles/GPT/struct_test.pickle_layer_{layer}', 'wb') as handle:
#         pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     print(f"Done writing data to pickle")
    
#     # Delete all lists so memory doesn't explode
#     del train_data
#     del dev_data
#     del test_data

# Train probe

In [ ]:
# Train the probe on GPT embeddings
epochs = 20

GPT_dev_loss, GPT_dev_uuas, GPT_test_loss, GPT_test_uuas = train(GPT_train_data, GPT_dev_data, GPT_test_data, epochs)

# Print the evaluation results
print("Dev Loss:", GPT_dev_loss[-1])
print("Dev UUAS:", GPT_dev_uuas[-1])
print("Test Loss:", GPT_test_loss)
print("Test UUAS:", GPT_test_uuas)

In [ ]:
# Train the probe on LSTM embeddings
epochs = 20

lstm_dev_loss, lstm_dev_uuas, lstm_test_loss, lstm_test_uuas = train(lstm_train_data, lstm_dev_data, lstm_test_data, epochs)

# Print the evaluation results
print("Dev Loss:", lstm_dev_loss[-1])
print("Dev UUAS:", lstm_dev_uuas[-1])
print("Test Loss:", lstm_test_loss)
print("Test UUAS:", lstm_test_uuas)

# Plot results

In [ ]:
import matplotlib.pyplot as plt

print("LSTM test Loss:", lstm_test_loss)
print("LSTM test UUAS:", lstm_test_uuas)
print("GPT test Loss:", GPT_test_loss)
print("GPT test UUAS:", GPT_test_uuas)

plt.plot([i for i in range(epochs)], lstm_dev_loss, label='LSTM')
plt.plot([i for i in range(epochs)], GPT_dev_loss, label='GPT')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss per Epoch")
plt.legend()
plt.show()

plt.plot([i for i in range(epochs)], lstm_dev_uuas, label='LSTM')
plt.plot([i for i in range(epochs)], GPT_dev_uuas, label='GPT')
plt.xlabel("Epochs")
plt.ylabel("UUAS score")
plt.title("UUAS per epoch")
plt.legend()
plt.show()

# Amnesic probing

**Helper functions**

In [ ]:
import numpy as np
import scipy

def get_probe_projection(probe):
    """
    Function to get the nullspace projection of a linear probe
    """

    null_space_proj = scipy.linalg.null_space(probe)
    return null_space_proj


def project_representations(data, probe):
    """
    Function to project all training data on the null space of the probe,
    as such effectively removing all information conveyed by the probe
    """
    null_space_proj = get_probe_projection(probe)
    projected_data = torch.tensor([elem[1]@null_space_proj for elem in data]) #TODO: do you need elem[1] here? or is elem just the embedding in itself already?

    return projected_data

**Cell for creating the structural data from the embeddings**

In [ ]:
def get_struct_data(path, embs):

    corpus = parse_corpus(path)
    gold_distances = create_gold_distances(corpus)
    
    return gold_distances, embs

**Actual amnesic probing**

In [ ]:
num_layers = 7
epochs = 20
test_UUAS_per_layer = []

model_name = "GPT"

for layer in num_layers:

    # Load POS probe data from pickle
    with open('pickles/' + model_name + f'/train_x_layer_{layer}.pickle', 'rb') as handle:
        GPT_train_x = pickle.load(handle)
    with open('pickles/' + model_name + f'/train_y_layer_{layer}.pickle', 'rb') as handle:
        GPT_train_y = pickle.load(handle)
    with open('pickles/' + model_name + f'/train_vocab_layer_{layer}.pickle', 'rb') as handle:
        GPT_train_vocab = pickle.load(handle)

    with open('pickles/' + model_name + f'/dev_x.pickle_layer_{layer}', 'rb') as handle:
        GPT_dev_x = pickle.load(handle)
    with open('pickles/' + model_name + f'/dev_y.pickle_layer_{layer}', 'rb') as handle:
        GPT_dev_y = pickle.load(handle)

    with open('pickles/' + model_name + f'/test_x.pickle_layer_{layer}', 'rb') as handle:
        GPT_test_x = pickle.load(handle)
    with open('pickles/' + model_name + f'/test_y.pickle_layer_{layer}', 'rb') as handle:
        GPT_test_y = pickle.load(handle)

    # # Big train set that is dev set and train set combined, as skorch does train/dev split itself
    # with open('pickles/' + model_name + '/train_x_big.pickle', 'rb') as handle:
    #     GPT_train_x_big = pickle.load(handle)
    # with open('pickles/' + model_name + '/train_y_big.pickle', 'rb') as handle:
    #     GPT_train_y_big = pickle.load(handle)

    POS_probe = LogisticRegression()
    POS_probe.fit(GPT_train_x, GPT_train_y)

    # Predict using the trained model
    GPT_train_predictions = POS_probe.predict(GPT_train_x)
    GPT_dev_predictions = POS_probe.predict(GPT_dev_x)
    GPT_test_predictions = POS_probe.predict(GPT_test_x)

    # Calculate accuracy scores
    GPT_train_accuracy = accuracy_score(GPT_train_y, GPT_train_predictions)
    GPT_dev_accuracy = accuracy_score(GPT_dev_y, GPT_dev_predictions)
    GPT_test_accuracy = accuracy_score(GPT_test_y, GPT_test_predictions)

    # print("Training accuracy:", GPT_train_accuracy)
    # print("Dev accuracy:", GPT_dev_accuracy)
    # print("Test accuracy:", GPT_test_accuracy)

    GPT_train_data = get_struct_data("data/en_ewt-ud-train.conllu", GPT_train_x)
    GPT_dev_data = get_struct_data("data/en_ewt-ud-dev.conllu", GPT_dev_x)
    GPT_test_data = get_struct_data("data/en_ewt-ud-test.conllu", GPT_test_x)
    # # Load the data for structural probing
    # with open('pickles/' + model_name + f'/struct_train_layer_{layer}.pickle', 'rb') as handle:
    #     GPT_train_data = pickle.load(handle)
    # with open('pickles/' + model_name + f'/struct_dev_layer_{layer}.pickle', 'rb') as handle:
    #     GPT_dev_data = pickle.load(handle)
    # with open('pickles/' + model_name + f'/struct_test_layer_{layer}.pickle', 'rb') as handle:
    #     GPT_test_data = pickle.load(handle)

    # Get the "amnesic vectors", the representations with POS information removed
    GPT_train_projected = project_representations(GPT_train_data, POS_probe)
    GPT_dev_projected = project_representations(GPT_dev_data, POS_probe)
    GPT_test_projected = project_representations(GPT_test_data, POS_probe)
    
    # Train the structural probe on these amnesic vectors
    _, _, proj_GPT_test_loss, proj_GPT_test_uuas = train(GPT_train_projected, GPT_dev_projected, GPT_test_projected, epochs)

    test_UUAS_per_layer.append(proj_GPT_test_uuas)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(7), test_UUAS_per_layer)
plt.xlabel("Layer in distilgpt2")
plt.ylabel("UUAS")
plt.show()

## LaTeX trees

For your report you might want to add some of those fancy dependency tree plots like those of Figure 2 in the Structural Probing paper. For that you can use the following code, that outputs the corresponding LaTeX markup.

**N.B.**: for the latex tikz tree the first token in a sentence has index 1 (instead of 0), so take that into account with the predicted and gold edges that you pass to the method.

In [ ]:
def print_tikz(predicted_edges, gold_edges, words):
    """ Turns edge sets on word (nodes) into tikz dependency LaTeX.
    Parameters
    ----------
    predicted_edges : Set[Tuple[int, int]]
        Set (or list) of edge tuples, as predicted by your probe.
    gold_edges : Set[Tuple[int, int]]
        Set (or list) of gold edge tuples, as obtained from the treebank.
    words : List[str]
        List of strings representing the tokens in the sentence.
    """

    string = """\\begin{dependency}[hide label, edge unit distance=.5ex]
    \\begin{deptext}[column sep=0.05cm]
    """

    string += (
        "\\& ".join([x.replace("$", "\$").replace("&", "+") for x in words])
        + " \\\\\n"
    )
    string += "\\end{deptext}" + "\n"
    for i_index, j_index in gold_edges:
        string += "\\depedge[-]{{{}}}{{{}}}{{{}}}\n".format(i_index, j_index, ".")
    for i_index, j_index in predicted_edges:
        string += f"\\depedge[-,edge style={{red!60!}}, edge below]{{{i_index}}}{{{j_index}}}{{.}}\n"
    string += "\\end{dependency}\n"
    print(string)